In [21]:
import pandas as pd
import geopandas

In [65]:
df = pd.read_csv('../data/predicted-favelas.csv')

df.head()

print(df.shape)

df = df.drop_duplicates()

df.shape

(27817, 3)


(26461, 3)

In [63]:
# df = pd.read_csv('../data/predicted-favelas_2.csv')

# df.head()

# print(df.shape)

# df_dups = df.drop_duplicates()

# df_dups.shape

(27816, 3)


(26460, 3)

(26461, 3)

In [61]:
27817 - 26461

1356

In [66]:
df_gp = pd.read_csv(f"../data/sp_geoprocessing.csv")

df_gp.shape

(27817, 41)

In [67]:
df_gp_dups = df_gp.drop_duplicates()
df_gp_dups.shape

(27817, 41)

In [68]:
df['cd_geocodi'].value_counts()

353440105000774    1
355030844000054    1
355030890000219    1
353440105000595    1
355030829000140    1
                  ..
355030895000110    1
354870805000767    1
355030845000208    1
355030838000150    1
355030859000177    1
Name: cd_geocodi, Length: 26461, dtype: int64

In [69]:
df[df['cd_geocodi'] == 355030815000006]

,cd_geocodi,favela_present,predicted_favela_present
9468,355030815000006,1.0,1.0


In [70]:
df['cd_geocodi'] = df['cd_geocodi'].astype(str) 

df.dtypes


cd_geocodi                   object
favela_present              float64
predicted_favela_present    float64
dtype: object

In [71]:
wd = '/Users/danielmsheehan/Dropbox/General_Assembly/Projects/project_5/data'


geo_df = geopandas.read_file(f'{wd}/maps/sp_census_tracts_wgs_84.shp')

geo_df.columns = [c.lower() for c in geo_df.columns]

geo_df = geo_df[['cd_geocodi', 'geometry']]

geo_df = geo_df.merge(df, on='cd_geocodi', how='left')



geo_df.dtypes
# gjson

print(geo_df.shape)

true_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 1) ]
false_neg = geo_df[(geo_df['favela_present'] == 0) & (geo_df['predicted_favela_present'] == 1) ]
false_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 0) ]

true_pos_geojson = true_pos.to_crs(epsg='4326').to_json()
false_neg_geojson = false_neg.to_crs(epsg='4326').to_json()
false_pos_geojson = false_pos.to_crs(epsg='4326').to_json()

true_pos.to_file(f'{wd}/maps/favela_preds_true_pos.shp')
false_neg.to_file(f'{wd}/maps/favela_preds_false_neg.shp')
false_pos.to_file(f'{wd}/maps/favela_preds_false_pos.shp')



(26461, 4)


In [89]:
real_favela = geopandas.read_file(f'{wd}/maps/sp_favelas_wgs_84.shp')
real_favela_geojson = real_favela.to_crs(epsg='4326').to_json()

In [37]:



print(geo_df.shape)


(5640, 5)


In [95]:
import folium

m = folium.Map(location=[-23.55, -46.533333], tiles="cartodbpositron", zoom_start=12)

#true_pos_geojson 
#false_neg_geojson 
#false_pos_geojson
def style_true_pos_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#66ff99'
    }

def style_false_neg_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#cc0099'
    }

def style_false_pos_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#ffcc00'
    }

def style_real_favela_geojson(feature):
    return {
        #'fillOpacity': 0.9,
        'weight': 4,
        'fillColor': '#66ff99',
        'color': '#001a09',
    }


folium.GeoJson(
    real_favela_geojson, 
    name='Real Favelas', 
    style_function=style_real_favela_geojson,
).add_to(m)


folium.GeoJson(
    true_pos_geojson, 
    name='True Positives', 
    style_function=style_true_pos_geojson,
).add_to(m)

folium.GeoJson(
    false_neg_geojson, 
    name='False Negatives',
    style_function=style_false_neg_geojson,
).add_to(m)

folium.GeoJson(
    false_pos_geojson, 
    name='False Positives',
    style_function=style_false_pos_geojson,
).add_to(m)



folium.LayerControl().add_to(m)

m.save('index.html')
#m

# Census Tracts

In [116]:
census_tracts = geopandas.read_file(f'{wd}/maps/sp_census_tracts_select_example_wgs_84.shp')
census_tracts_geojson = census_tracts.to_crs(epsg='4326').to_json()

favelas = geopandas.read_file(f'{wd}/maps/sp_favelas_wgs_84.shp')
favelas_geojson = favelas.to_crs(epsg='4326').to_json()


m = folium.Map(location=[-23.65, -46.59033333], tiles="cartodbpositron", zoom_start=12)


def style_census_tracts_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 4,
        'fillColor': '#3385ff',
        'color': '#002966',
    }

def style_favelas_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 4,
        'fillColor': '#66ff99',
        'color': '#001a09',
    }


# def style_false_pos_geojson(feature):
#     return {
#         'fillOpacity': 0.8,
#         'weight': 0,
#         'fillColor': '#ffcc00'
#     }

# def style_real_favela_geojson(feature):
#     return {
#         #'fillOpacity': 0.9,
#         'weight': 4,
#         'fillColor': '#66ff99',
#         'color': '#001a09',
#     }


folium.GeoJson(
    census_tracts_geojson, 
    name='Census Tracts Example', 
    style_function=style_census_tracts_geojson,
).add_to(m)


folium.GeoJson(
    favelas_geojson, 
    name='Real Favelas', 
    style_function=style_favelas_geojson,
).add_to(m)

# folium.GeoJson(
#     false_neg_geojson, 
#     name='False Negatives',
#     style_function=style_false_neg_geojson,
# ).add_to(m)

# folium.GeoJson(
#     false_pos_geojson, 
#     name='False Positives',
#     style_function=style_false_pos_geojson,
# ).add_to(m)



folium.LayerControl().add_to(m)

m.save('data.html')
